# Ensemble

In [ ]:
%load_ext autoreload
%autoreload 2

import json
from copy import deepcopy
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import ceil
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils
from IPython.display import display, clear_output
import glob
import cv2
import pickle5 as pickle
from multiprocessing import Pool
from itertools import repeat
from utils import get_paths, process_img_for_ensemble

## Execution parameters

In [ ]:
num_cpus = 24  # Number of CPUs to be used
base_dir = ''  # Specify full path to provided directory 'final_ensemble_models_and_results'
img_dir = ''  # Specify full path to 'x' directory of SegPC final test set images

export_flag = True  # Set to False to disable the creation of images

### No need to modify anything below this point

In [ ]:
output_dir = os.path.join(base_dir, 'result_images')
if export_flag:
    print('Exporting images to:', output_dir)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

In [ ]:
model_codes = [
    'scnet_r50_20e',
    'scnet_s50_1x',
    'scnet_r101_20e',
    'htc_r50_1x',
    'htc_r101_20e',
    'scnet_s101_1x',
    'htc_s101_1x'
]

epochs = ['epoch_13', 'epoch_5', 'epoch_12', 'epoch_11', 'epoch_14', 'epoch_3', 'epoch_2']

iou_thold = 0.85

cat_ids = {
    'nucleus_id': 0,
    'cytoplasm_id': 1,
    'cell_id': 2,
}

initial_score_thold = 0  # Score threshold to directly filter out some nucleus and cytoplasm candidates

In [ ]:
model_results_dict = {k: None for k in model_codes}
model_paths_dict = {k: None for k in model_codes}

for model_code, epoch in zip(model_codes, epochs):
    model_paths_dict[model_code] = get_paths(img_dir, base_dir, model_code, epoch)
    
    with open(model_paths_dict[model_code]['all_results_df_file'], "rb") as fh:
          model_results_dict[model_code] = pickle.load(fh)        

In [ ]:
ref_model_code = model_codes[0]
other_models_codes = model_codes[1:]

cocoGt = COCO(model_paths_dict[ref_model_code]['ann_file'])
result_anns = json.load(open(model_paths_dict[ref_model_code]['segm_result_file']))

all_results_df = pd.DataFrame()


processing_pool = Pool(num_cpus)

img_ids = cocoGt.getImgIds()
results = processing_pool.starmap(process_img_for_ensemble, zip(img_ids,
                                                   repeat(cocoGt),
                                                   repeat(model_paths_dict),
                                                   repeat(model_results_dict),
                                                   repeat(ref_model_code),
                                                   repeat(other_models_codes),
                                                   repeat(iou_thold),
                                                   repeat(export_flag),
                                                   repeat(output_dir)
                                                  ))

for result in results:
    all_results_df = pd.concat([all_results_df, result], axis=0, ignore_index=True)

processing_pool.close()